# IMPORTS

In [2]:
import keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Reshape, Activation, ReLU, \
    Add, GlobalMaxPooling2D, Input
from keras.models import Sequential
from keras.optimizers import Adadelta, Adam, Adagrad
from sklearn.metrics import confusion_matrix
from keras.losses import binary_crossentropy
from keras import Model

# TRAIN DATA BEFORE SMOTE

In [3]:
# from utils.read_cnn import read_train
#
# train_images, train_labels = read_train()
#
# unique_labels, label_counts = np.unique(train_labels, return_counts=True)
#
# plt.bar(unique_labels, label_counts)
#
# plt.xticks(unique_labels, unique_labels)
#
# plt.title('Label Distribution')
# plt.xlabel('Label')
# plt.ylabel('Count')
# plt.show()

# TRAIN DATA AFTER SMOTE

In [ ]:
from utils.read_cnn import read_train_smote

train_images_smote, train_labels_smote = read_train_smote(slice_value=10000)

print(train_images_smote.shape)

unique_labels, label_counts = np.unique(train_labels_smote, return_counts=True)

plt.bar(unique_labels, label_counts)

plt.xticks(unique_labels, unique_labels)

plt.title('Label Distribution')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()

In [ ]:
from skimage import io

img_array = train_images_smote[np.random.randint(0, train_images_smote.shape[0])]

# Display the image from the numpy array using scikit-image's imshow function
io.imshow(img_array.squeeze(), cmap='gray')
io.show()

# VALIDATION DATA

In [6]:
from utils.read_cnn import read_validation

validation_images, validation_labels = read_validation()

# F1 SCORE METRIC

In [7]:
def score(y_true, y_pred):  #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

# CALLBACKS

In [8]:
learn_rate = ReduceLROnPlateau(monitor='loss', factor=0.3, patience=2, verbose=1, mode='auto', min_delta=0.00005,
                               cooldown=1, min_lr=0.0001)

early_stop = EarlyStopping(monitor='val_score', min_delta=0.0001, patience=30, verbose=1, mode='max', baseline=None,
                           restore_best_weights=True)

# MODEL

In [9]:
from time import time
from utils.read_cnn import image_size

start = time()

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(image_size[0], image_size[1], 1)))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Conv2D(filters=32, kernel_size=5, padding='same', activation='relu', strides=2))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Conv2D(filters=64, kernel_size=5, padding='same', activation='relu', strides=2))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss=binary_crossentropy, metrics=[score])

model.summary()

history = model.fit(train_images_smote, train_labels_smote, epochs=500, batch_size=32, verbose=1,
                    validation_data=(validation_images, validation_labels),
                    callbacks=[early_stop])

model.save('models/cnn_model.h5')

print(f"--------------------------- {time() - start} ---------------------------")

# PLOT

In [ ]:
plt.figure()
plt.plot(history.history['score'])
plt.plot(history.history['val_score'])
plt.title('Model Score')
plt.ylabel('F1 Score')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# CONFUSION MATRIX

In [ ]:
import itertools


def show(matrix, type):
    classes = [0, 1]
    plt.imshow(matrix, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion Matrix " + type)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = 'd'
    thresh = matrix.max() / 2.
    for i, j in itertools.product(range(matrix.shape[0]), range(matrix.shape[1])):
        plt.text(j, i, format(matrix[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if matrix[i, j] > thresh else "black")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

# CONFUSION MATRIX TRAIN

In [ ]:
train_predicted_labels = model.predict(train_images_smote, verbose=1)
train_predicted_labels = np.round(train_predicted_labels).astype(int).reshape(-1, )
cm = confusion_matrix(train_labels_smote, train_predicted_labels)
show(cm, "Train")

# CONFUSION MATRIX VALIDATION

In [ ]:
val_predicted_labels = model.predict(validation_images, verbose=1)
val_predicted_labels = np.round(val_predicted_labels).astype(int).reshape(-1, )
cm = confusion_matrix(validation_labels, val_predicted_labels)
show(cm, "Validation")

# TEST DATA

In [ ]:
from utils.read_cnn import read_test

test_images = read_test()

# TEST

In [ ]:
from utils.write import write

predicted_labels = model.predict(test_images, verbose=1)
predicted_labels = np.round(predicted_labels).astype(int).reshape(-1, )

write(predicted_labels)

print(f"-----------------------------------------------------")